# State Vector Machines

## Introduction

## Summary of Results

---

## Setup 

First, we'll need to import the various libraries that we'll need.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

## Load and Clean Data

Before we can build any models, we need to import the data and clean it by converting types as necessary.

In [ ]:
df = pd.read_csv("Adult/adult.data", names=[
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "earning_label"
], skipinitialspace=True)

df

In [ ]:
# Explore some attributes of the dataset

# dir(df)
print("Features: ", df.columns)
print("Labels: ", pd.Series.unique(df.earning_label))
print("Shape: ", df.shape)

In [ ]:
# Because the native.country variable has too many categories, and most of the data points are from the US (91%), we combine all the categories except for “United-States”into the “Other” category:
df.loc[df['native_country']!='United-States', 'native_country']='Other'

# Save the output label in binary encoding, 0: <=50k, 1: > 50k
Y=pd.Categorical(df['earning_label']).codes
Y = np.where(Y==0, -1, Y) 
print(Y)

# education is not needed as uducation_num performs its function
# also drop the label as it is not needed for the model
df=df.drop(['education','earning_label'], axis=1)

# determine unique values of each categorical feature:
for feature in ['workclass','marital_status','occupation','relationship','race','sex','native_country']:
    print(feature, pd.Series.unique(df[feature]))

df

In [ ]:
# Convert categorical features to one-hot encoding
df=pd.get_dummies(df)
df

# encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
# encoder.fit(df)
# df = encoder.transform(df)
# df

In [ ]:
# Finally, split the data into training and testing sets 80\/20
train_X, test_X, train_Y, test_Y = train_test_split(df.values, Y, test_size = 0.2)

## Fitting the Model

With the data now processed, it is ready to have SVM applied

## Acknowlegements

https://methods.sagepub.com/dataset/howtoguide/support-vector-machine-in-aci-1996-python